In [1]:
import pandas as pd
import json
import requests

In [2]:
response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/incidentes2019.json")
incidents_db = json.loads(response.text)
response1 = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/distancias_estaciones_barrios.json")
distances_stations_ntas_db = json.loads(response1.text)
neighbors = distances_stations_ntas_db["42 South Street"].keys()

In [3]:
ev = 197
lv = 143
rv = 5
sv = 8
hv = 1

In [4]:
df =pd.DataFrame()

In [5]:
nta = [elem["nta"] for elem in incidents_db]
units = [elem["units"] for elem in incidents_db]
incident_duration = [elem["incident_duration"] for elem in incidents_db]
first_shift = [elem["is_first_shift"] for elem in incidents_db]
df["nta"]=nta
df["units"]=units
df["incident_duration"]=incident_duration
df["first_shift"]=first_shift

In [6]:
f_shift_db = df[df["first_shift"]==True].reset_index().drop(["index"],axis=1)
s_shift_db = df[df["first_shift"]==False].reset_index().drop(["index"],axis=1)

In [90]:
f_shift_db

,nta,units,incident_duration,first_shift
0,MN03,[ladder],779.0,True
1,MN23,[engine],1935.0,True
2,BK43,[ladder],521.0,True
3,BX14,[engine],1206.0,True
4,BK33,[ladder],730.0,True
...,...,...,...,...
59360,QN26,[ladder],339.0,True
59361,BX44,[engine],710.0,True
59362,MN23,"[engine, ladder, ladder, ladder, engine, engine]",2555.0,True
59363,QN62,[rescue],1860.0,True


In [7]:
units_per_nta_per_shift1 = f_shift_db.groupby(by="nta")["units"].sum()
units_per_nta_per_shift2 = s_shift_db.groupby(by="nta")["units"].sum()

In [8]:
n1 = list(units_per_nta_per_shift1.index)
n2 = list(units_per_nta_per_shift2.index)

In [9]:
ladders = 0
engines = 0
squads = 0
rescues = 0
ha = 0
for elem in units_per_nta_per_shift1:
    
    for e in elem:
        if e == "ladder":ladders+=1
        elif e == "squad": squads+=1
        elif e == "engine": engines+=1
        elif e == "hazardous": ha+=1
        else : rescues+=1

In [10]:
print(ladders)
print(squads)
print(engines)
print(rescues)
print(ha)

56203
2415
47109
4702
2319


In [11]:
ntas = []
engines_vehicles = []
squad_vehicles = []
rescue_vehicles=[]
ha_vehicles = []
ladder_vehicles = []

for i,elem in enumerate(units_per_nta_per_shift1):
    l = elem.count('ladder')
    e = elem.count('engine')
    h = elem.count('hazardous')
    s = elem.count('squad')
    r = elem.count('rescue')
    
    s_l = l/ladders
    s_e = e/engines
    s_h = h/ha
    s_s = s/squads
    s_r = r/rescues

    ntas.append(n1[i])
    engines_vehicles.append(s_e*ev)
    rescue_vehicles.append(s_r*rv)
    squad_vehicles.append(s_s*sv)
    ha_vehicles.append(s_h*hv)
    ladder_vehicles.append(s_l*lv)



    

In [12]:
ladders2 = 0
engines2 = 0
squads2 = 0
rescues2 = 0
ha2 = 0
for elem in units_per_nta_per_shift2:
    
    for e in elem:
        if e == "ladder":ladders2+=1
        elif e == "squad": squads2+=1
        elif e == "engine": engines2+=1
        elif e == "hazardous": ha2+=1
        else : rescues2+=1

In [13]:
for i,elem in enumerate(units_per_nta_per_shift2):
    l = elem.count('ladder')
    e = elem.count('engine')
    h = elem.count('hazardous')
    s = elem.count('squad')
    r = elem.count('rescue')
    
    s_l = l/ladders
    s_e = e/engines
    s_h = h/ha
    s_s = s/squads
    s_r = r/rescues

    ntas.append(n2[i])
    engines_vehicles.append(s_e*ev)
    rescue_vehicles.append(s_r*rv)
    squad_vehicles.append(s_s*sv)
    ha_vehicles.append(s_h*hv)
    ladder_vehicles.append(s_l*lv)




In [14]:
df_final = pd.DataFrame()
df_final["nta"]= ntas
df_final["engines"]=engines_vehicles
df_final["squads"]=squad_vehicles
df_final["ladders"]=ladder_vehicles
df_final["ha"]=ha_vehicles
df_final["rescue"]=rescue_vehicles
df_final["shift"]=[1]*45+[0]*44

In [15]:
df_final.to_csv("/home/vant/Documentos/mySpace/cuarto/opt/projectI/Optimization_NY_Fire_Departments/aux_data/bijk.csv")